<a href="https://colab.research.google.com/github/RSBalchII/project-1-transaction-data/blob/main/project_1_transaction_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 1: Transaction Success Prediction

By: Robert S. Balch II

## Problem Definition: #1

This project is related to a binary classification problem using user historical data to predict whether or not a customer will make a transaction in the future based on their past transaction data.

The data is available on AWS S3 at https://ddc-datascience.s3.amazonaws.com/Projects/Project.1-Transactions/Data/Transaction.train.csv


## Data Collection/Imports

### #2 Load Imports

In [408]:
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

from sklearn import datasets, metrics, model_selection
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score

### #3 Load train.csv

In [409]:
base_path = "https://ddc-datascience.s3.amazonaws.com/Projects/Project.1-Transactions/Data/Transaction.train.csv"
train_path = base_path + 'Transaction.train.csv'
test_path = base_path + 'Testing.test.csv'

In [410]:
train_path

'https://ddc-datascience.s3.amazonaws.com/Projects/Project.1-Transactions/Data/Transaction.train.csvTransaction.train.csv'

## Data Cleaning

### #4 Examine Data

In [411]:
trans_data = pd.read_csv(base_path)

#### create default data frame

In [412]:
trans_data_df = pd.DataFrame(trans_data)

In [413]:
trans_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180000 entries, 0 to 179999
Data columns (total 53 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  180000 non-null  int64  
 1   ID_code     180000 non-null  object 
 2   target      180000 non-null  int64  
 3   var_0       180000 non-null  float64
 4   var_1       180000 non-null  float64
 5   var_2       180000 non-null  float64
 6   var_3       180000 non-null  float64
 7   var_4       180000 non-null  float64
 8   var_5       180000 non-null  float64
 9   var_6       180000 non-null  float64
 10  var_7       180000 non-null  float64
 11  var_8       180000 non-null  float64
 12  var_9       180000 non-null  float64
 13  var_10      180000 non-null  float64
 14  var_11      180000 non-null  float64
 15  var_12      180000 non-null  float64
 16  var_13      180000 non-null  float64
 17  var_14      180000 non-null  float64
 18  var_15      180000 non-null  float64
 19  va

## #5 Determine Data Cleaning

In [414]:
trans_data_df.sample(n = 100)

,Unnamed: 0,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,...,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49
120451,120451,train_120451,0,11.2811,4.7879,9.6031,6.6646,8.8750,14.2753,15.1446,...,5.6793,-4.8303,-10.6003,24.1287,-1.5085,19.0707,-1.4546,11.0212,3.9427,15.2486
49957,49957,train_49957,0,16.1763,-6.0137,8.3845,5.6125,8.1183,14.2124,11.6687,...,6.2236,0.3856,4.7783,25.0809,0.9155,20.4802,1.2372,8.2602,1.6525,18.2520
115072,115072,train_115072,0,11.5778,4.0513,11.0615,5.9291,8.5484,13.8261,10.0410,...,6.2507,3.1608,-4.4164,17.7447,1.9091,23.0961,-0.5887,11.6499,0.9968,19.1727
108410,108410,train_108410,0,8.8475,-2.9646,12.9073,4.2131,7.8394,13.5542,4.6399,...,5.4835,-7.7614,6.6790,20.1156,5.1390,34.8426,9.3475,5.7787,-1.3631,9.1764
11225,11225,train_11225,0,13.4892,-4.6375,14.0525,4.8566,7.0591,13.9706,9.0683,...,5.6321,0.9122,5.8129,29.6557,2.3944,16.7830,0.5090,5.5231,3.9902,17.9611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113262,113262,train_113262,0,8.2047,0.4654,14.2480,6.3029,7.9472,13.9818,5.5766,...,5.1907,4.7215,8.3551,11.5238,0.0243,8.1783,5.6326,9.4474,1.4296,16.5557
77041,77041,train_77041,0,4.6997,1.3306,9.7885,5.2227,7.0191,14.1866,10.8823,...,5.3677,-1.1919,-3.0133,11.2691,8.5292,28.9222,-0.7473,13.2256,0.5352,20.9588
140384,140384,train_140384,1,13.9833,1.9758,11.9405,4.4079,9.4809,14.0046,4.0410,...,5.7621,-4.8409,-5.3604,28.7248,8.0450,19.2577,-1.6456,7.3530,0.0148,16.3762
92232,92232,train_92232,0,12.7798,2.6713,13.7355,6.0255,7.1366,13.6848,1.8151,...,5.0159,-5.0506,10.5710,20.8262,5.7932,16.4795,10.9472,2.6470,0.6851,20.0462


## My Cleaning Recommendations
Unnamed: 0 and ID_code appear to have unusable data.
- Unnamed: 0 data is simply listing the position of the row which isnt helpful for our purposes
- ID_code is "object" data type

## Copy of trans_data_df
= trans_data_copy to trans_data_copy_2

In [415]:
trans_data_copy = trans_data_df.copy()

#### drop columns that seem to have un-corroborative data

In [416]:
trans_data_drop1 = trans_data_copy.drop("Unnamed: 0", axis=1)

In [417]:
trans_data_drop2 = trans_data_drop1.drop("ID_code", axis=1)

In [418]:
trans_data_copy_2 = trans_data_drop2.copy()

#### check for null values

In [419]:
trans_data_copy_2.isnull().sum()

,0
target,0
var_0,0
var_1,0
var_2,0
var_3,0
var_4,0
var_5,0
var_6,0
var_7,0
var_8,0


#### Mean, Median, Mode

In [420]:
trans_data_copy_2.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49
count,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.00000,180000.000000,180000.000000,...,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000
mean,0.100222,10.678650,-1.630045,10.712922,5.408688,7.566953,14.024017,8.52808,15.243424,17.264876,...,5.635897,-0.000418,-0.668334,20.208845,2.595090,12.756574,3.236201,7.440715,1.927078,15.870166
std,0.300297,3.041036,4.050519,2.639146,0.866265,1.235043,0.190038,4.64170,7.851050,8.203204,...,0.369629,4.422915,5.968300,7.132446,2.801692,9.320939,4.561317,3.021636,1.476777,3.010492
min,0.000000,0.408400,-15.043400,2.117100,2.347300,3.970500,13.434600,-6.01110,-10.664200,-10.089000,...,4.258800,-14.506000,-22.748700,-2.995300,-7.856100,-18.489700,-14.093300,-2.691700,-3.814500,6.299300
25%,0.000000,8.451675,-4.745600,8.720000,4.768500,6.618800,13.893900,5.06575,9.170200,11.552000,...,5.374200,-3.253575,-4.997500,15.062900,0.628600,5.661300,-0.062000,5.160325,0.890000,13.830400
50%,0.000000,10.526750,-1.611250,10.579600,5.385600,7.628500,14.025500,8.59920,15.198900,17.246850,...,5.634500,0.008500,-0.560300,20.203300,2.513800,12.520300,3.204650,7.350950,1.899700,15.934600
75%,0.000000,12.756200,1.360225,12.514100,6.003200,8.583800,14.164400,12.27330,21.010500,23.100050,...,5.905600,3.098325,3.632525,25.639125,4.396725,19.461725,6.409900,9.511125,2.947225,18.061425
max,1.000000,20.315000,10.376800,19.353000,8.447700,11.150600,14.654500,22.33150,41.748000,49.044300,...,7.089000,16.731900,18.855400,43.546800,13.448700,44.363400,18.440900,16.716500,8.402400,26.079100


## Data Exploration

 #6 In the following cells I have visualized the cleaned dataframe across each column on.
 #7 I verified that the target column had no null values.
 #8

### #6 Visual Analysis of data

In [ ]:
trans_data_copy_2.hist(figsize=(13, 13), bins=100)
plt.tight_layout()
plt.show()

### #7 Verify 'target' column

In [ ]:
trans_data_copy_2.info()

## #8 Check Correlations between predictors

### View correlation graphs and data

In [ ]:
# Here we show a heatmap of the correlations in this remaining data

# corr calculates the correlation matrix for the dataset in trans_data_copy_2
corr = trans_data_copy_2.corr()
# here the heatmap is generated and sized
plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap='magma',annot = False);

In [ ]:
# Here we removes the perfect correlations so we can foCus on the relationships between different unique variables
corr_unstacked = corr.unstack()
corr_unstacked

In [ ]:
# Here we select correlations with the 'target' variable and take the absolute values of these correlations
corr_unstacked_not_self = corr_unstacked[ corr_unstacked !=1 ]
corr_unstacked_not_self

In [ ]:
corr_unstacked_not_self['target'].abs().sort_values(ascending = False)

In [ ]:
corr_pair = corr_unstacked_not_self.abs().sort_values(kind='quicksort', ascending = False)
corr_pair

In [ ]:
corr_pair[::2].head(20)

## Data Processing

### #9 Create sale and no_sale Dataframes

In [ ]:
df_target_0 = trans_data_copy_2[trans_data_copy_2['target'] == 0]

In [ ]:
df_target_0.shape

In [ ]:
df_target_1 = trans_data_copy_2[trans_data_copy_2['target'] == 1]

In [ ]:
df_target_1.shape

In [ ]:
sale = pd.DataFrame(df_target_1).copy()

In [ ]:
no_sale = pd.DataFrame(df_target_0).copy()

### #10 predictor and target Dataframes

In [ ]:
drop_target = trans_data_copy_2.copy()

In [ ]:
dropped_target = drop_target.drop("target", axis=1)

In [ ]:
predictors = dropped_target.copy()

In [ ]:
drop_var = trans_data_copy_2.copy()

In [ ]:
columns_to_drop = [col for col in drop_var.columns if col != 'target']

In [ ]:
dropped_var = drop_var.drop(columns=columns_to_drop)

In [ ]:
target = dropped_var
target

### #11 initialize GaussianNB()

In [ ]:
gnb = GaussianNB()

### create train_test_split

### #12 Divide Data Frames
### #13 Train Model on testing subset
### #14 Train Model on training subset
### #15 cross-validation loop to calculate model accuracy

In [ ]:
X = predictors
y = target['target']

In [ ]:
numLoops = 50
accuracy_scores = np.zeros(numLoops)

for idx in range(numLoops):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    gnb.fit(X_train, y_train)
    y_pred = gnb.predict(X_test)
    accuracy_scores[idx] = metrics.accuracy_score(y_test, y_pred)

mean_accuracy = accuracy_scores.mean()
print(f"Mean accuracy over {numLoops} iterations: {mean_accuracy:.4f}")


In [ ]:
def cross_validate(X, y, num_loops):
  """Randomly splits X and y values into train/test groups (test size = 20%).
  Creates a GaussianNB model.
  Returns a numpy array of accuracy scores for the tests.
  """

  accuracy_scores = np.zeros(num_loops)

  for i in range(num_loops):
    model_split_cv = GaussianNB()
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)
    model_split_cv.fit(X_train, y_train)
    y_split_pred = model_split_cv.predict(X_test)
    accuracy_scores[i] = metrics.accuracy_score(y_test, y_split_pred)

  return accuracy_scores

In [ ]:
accuracy_scores = cross_validate(X, y, 100)
print(round(accuracy_scores.mean(), 5))

### #16 Histogram

In [ ]:
plt.hist(accuracy_scores)
plt.title('Frequency of accuracy scores for cross-validation')
plt.show()

### #17 Confusion Matrix

In [ ]:
metrics.ConfusionMatrixDisplay.from_estimator(
    gnb,
    X_test,
    y_test,
    normalize = 'true',
    values_format = '.2%',
    cmap = 'magma',
    )
plt.grid(False)
plt.title('Confusion matrix') ;
plt.savefig('confusion_matrix.png')

### #18 Remove enough no_sale transaction rows to get a 50% split on the remaining data

In [ ]:
# Calculate the number of successful transactions
num_successful = sale.shape[0]

# Calculate the number of unsuccessful transactions needed for a 50/50 split
num_unsuccessful_needed = num_successful

# Randomly sample the required number of unsuccessful transactions
balanced_unsuccessful = no_sale.sample(n=num_unsuccessful_needed, random_state=42)

# Combine the successful and balanced unsuccessful transactions
balanced_data = pd.concat([sale, balanced_unsuccessful])

In [ ]:
# Shuffle the balanced data to mix successful and unsuccessful transactions
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
X = balanced_data.drop('target', axis=1)
y = balanced_data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### #19 cross_validate this dataset with previous dataset

In [ ]:
num_successful = sale.shape[0]
num_successful

### #20 Compare the results of original model with balanced_data model

In [ ]:
num_unsuccessful_needed = num_successful

In [ ]:
balanced_unsuccessful = no_sale.sample(n=num_unsuccessful_needed, random_state=42)
balanced_unsuccessful

In [ ]:
whole_data_accuracy = trans_data_copy_2

In [ ]:
whole_mean_accuracy = round(accuracy_scores.mean(), 5)
balanced_mean_accuracy = round(balanced_data.mean(), 5)
whole_mean_accuracy


In [ ]:
balanced_data = pd.concat([sale, balanced_unsuccessful])
balanced_data

In [ ]:
X_balanced = balanced_data.drop('target', axis=1)
y_balanced = balanced_data['target']

X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

balanced_data_accuracy = cross_validate(X_balanced, y_balanced, 100)
balanced_data_mean_accuracy = round(balanced_data_accuracy.mean(), 5)
print(f"Mean accuracy on balanced dataset: {balanced_data_mean_accuracy}")

In [ ]:
X_whole = predictors
y_whole = target['target']

whole_data_accuracy = cross_validate(X_whole, y_whole, 100)
whole_data_mean_accuracy = round(whole_data_accuracy.mean(), 5)
print(f"Mean accuracy on whole dataset: {whole_data_mean_accuracy}")

### #21 Data Visualization

### Confusion Matrix and results of classification report

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.bar(['Whole Dataset', 'Balanced Dataset'], [whole_data_mean_accuracy, balanced_data_mean_accuracy])
plt.xlabel('Dataset')
plt.ylabel('Mean Accuracy')
plt.title('Cross-Validation Accuracy Comparison')
plt.show()

# Final thoughts/ Communicate Results

### Data Cleaning and Exploration:
The initial dataset contains 53 columns. Upon examination, the 'Unnamed: 0' and 'ID_code' columns appear irrelevant and are removed. The remaining data is checked for null values, and descriptive statistics are calculated. Histograms are generated for visual analysis of the data distribution. The 'target' column, representing transaction success (1) or failure (0), is verified.

###Correlation Analysis:
A correlation matrix is created to identify relationships between predictors. The analysis focuses on correlations with the 'target' variable, revealing the predictors with the prediction

### Model Training and Evaluation (Initial):
A Gaussian Naive Bayes (GNB) model is initialized. The data is split into training and testing sets. The model is trained and evaluated using a loop with 50 iterations, achieving a mean accuracy of 91.11%. Cross-validation is performed with 100 iterations, resulting in a mean accuracy of 75.857%.

### Addressing Class Imbalance:
The analysis recognizes a class imbalance in the dataset. To address this, a balanced dataset is created by randomly sampling an equal number of successful and unsuccessful transactions.

### Model Training and Evaluation (Balanced Data):
The GNB model is trained and evaluated on the balanced dataset. The mean accuracy on the balanced dataset is 75.8%, compared to 91.111% on the original dataset. This difference highlights the impact of class imbalance on model performance.

### Results and Comparison:
A bar chart visualizes the mean accuracy scores for both the original and balanced datasets. The confusion matrix and a classification report are generated to further evaluate the model's performance.

# Hypothesis:

The data suggests that the median chance of a customer making a purchase in the future is approximately 1/10. However, upon closer inspection, we observe slightly higher correlations between certain var_* variables and successful transactions.

Specifically:

The three highest correlated variables (var_20, var_31, and var_5) show correlation coefficients ranging from 0.080689 to 0.070015. While these correlations are not extremely strong, they do indicate a positive relationship between these variables and successful transactions.

Variables with correlation coefficients greater than 0.05 suggest a higher chance of a successful transaction. This threshold, while not definitive, provides a baseline for identifying potentially influential factors.

The correlation analysis reveals a pattern where certain variables are more strongly associated with successful purchases than others. This suggests that focusing on these variables could potentially improve predictive models and marketing strategies.

It's important to note that correlation does not imply causation and to keep in mind that the original dataset is highly imbalanced meaning the model chooses the majority class most of the time.

These relationships may indicate underlying factors that contribute to successful transactions, but further analysis would be needed to determine causal relationships.

The relatively low correlation coefficients across all variables suggest that successful transactions are likely influenced by a complex interplay of factors, rather than being strongly determined by any single variable.

This analysis provides valuable insights into the relationships between various factors and successful transactions, allowing for more informed decision-making in product development, marketing strategies, and customer targeting. However, it's crucial to interpret these findings in the context of broader market research and business knowledge to derive actionable insights.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for var in ['var_20', 'var_31', 'var_5']:
    sns.boxplot(x='target', y=var, data=trans_data_copy_2)
    plt.title(f'Distribution of {var} by Target')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X, y)
feature_importances = pd.Series(rf.feature_importances_, index=X.columns)
print(feature_importances.sort_values(ascending=False).head(10))


# Citations
[Titanic Survival Project:](https://nbviewer.org/urls/ddc-datascience.s3.us-west-1.amazonaws.com/Projects/Example/Titanic.Survival.Prediction.Using.Naive.Bayes.ipynb)
/
[Kyla Bendt:](https://github.com/KylaBendt)
